# SMorph: Group Analysis

---
## Setup
Please execute the cell(s) below to initialize the notebook environment.

In [ ]:
# @title Install dependencies
# !pip install poetry

In [ ]:
# @title Install SMorph Python module
# !pip install https://github.com/swanandlab/SMorph/releases/download/0.1.0/SMorph-0.1.0.tar.gz

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Suppress annoying warnings

import smorph as sm
import pandas as pd
import ipywidgets as widgets
import winsound
pd.set_option('display.max_rows', None)  # remove upper limit on display of rows

---
## Step 1: Define groups and image preprocessing parameters

- Set `LABELS` as list of strings containing name labels of each group
- Set `GROUP_FOLDERS` as list of strings containing paths to each group folder
- Set `IMG_TYPE` as string to select the image acquisition method
- Set `CROP_TECH` as string to select the cropping method used to crop out cells from tissue image

In [ ]:
# input the path to individual group folders (place them in SMorph)
GROUP_FOLDERS = ['Autocropped/SAL_28_MLUB_3D',
'Autocropped/DMI_28_MLUB_3D', 'Autocropped/FLX_28_MLUB_3D']  #@param

SCALE = (1.0785801681301463, 0.6918881978764917, 0.6918881978764917)

# group labels
LABELS = None  #@param

In [ ]:
# @title Interactive image parameter selection
IMG_TYPE = 'confocal'
SEGMENTED = False

def select_image_params (img_type, segmented=SEGMENTED):
  global IMG_TYPE, SEGMENTED
  IMG_TYPE = img_type
  SEGMENTED = segmented

_ = widgets.interact(select_image_params, img_type=['confocal', 'DAB'], segmented=[True, False])

Cell image preprocessing parameters:
* `min_ptile` and `max_ptile`: minimum and maximum contrast percentiles to stretch the image to
* `threshold_method`: method for single intensity auto-thresholding the cell image

Overlayed contour represents the thresholding results.

In [ ]:
# @title Interactive preprocessing parameter selection

CONTRAST_PTILES = (0, 100)
THRESHOLD_METHOD = None # sm.util.THRESHOLD_METHODS[6]

import matplotlib.pyplot as plt
import skimage.io as io
from random import choice
from os import listdir
from skimage.measure import find_contours
from copy import copy

my_cmap = copy(plt.cm.get_cmap('gray')) # get a copy of the gray color map
my_cmap.set_bad(alpha=0) # set how the colormap handles 'bad' values

rand_group_path = choice(GROUP_FOLDERS)
rand_img = choice(listdir(rand_group_path))
rand_img = f'{rand_group_path}/{rand_img}'
cell_image = io.imread(rand_img)
if cell_image.ndim == 3:
  cell_image = cell_image.max(2)
print(rand_img)

def plot_ptiles (
  min_ptile=CONTRAST_PTILES[0],
  max_ptile=CONTRAST_PTILES[1],
  threshold_method=THRESHOLD_METHOD,
  threshold_value=0
):
  global CONTRAST_PTILES, THRESHOLD_METHOD
  CONTRAST_PTILES = (min_ptile, max_ptile)
  THRESHOLD_METHOD = threshold_method
  if threshold_method is None:
    THRESHOLD_METHOD = threshold_value
  plt.imshow(sm.util._image._contrast_stretching(cell_image,
                                                 (min_ptile, max_ptile)),
             cmap='gray')
  mask = sm.util.preprocess_image(
    cell_image,
    IMG_TYPE,
    None,
    SEGMENTED,
    CONTRAST_PTILES,
    THRESHOLD_METHOD
  )[1]
  # contours = find_contours(mask, .9)
  # for contour in contours:
  #   plt.plot(contour[:, 1], contour[:, 0], linewidth=2)
  mask[~mask] = None # insert 'bad' values into your lattice
  plt.imshow(mask, alpha=.5, cmap=my_cmap)

_ = widgets.interact(plot_ptiles, min_ptile=(0, 100, 1),
  max_ptile=(0, 100, 1),
  threshold_method=[*sm.util.THRESHOLD_METHODS, None],
  threshold_value=(0., .99, .01))

---
## Step 2: Start group analysis

Sholl analysis parameters:
- Set `SHOLL_STEP_SIZE` as int to difference (in pixels) between concentric Sholl circles
- Set `POLYNOMIAL_DEGREE` as int to degree of polynomial for fitting regression model on Sholl values

In [ ]:
SHOLL_STEP_SIZE = 3  #@param
POLYNOMIAL_DEGREE = 3  #@param

In [ ]:
groups = sm.Groups(GROUP_FOLDERS, image_type=IMG_TYPE, scale=SCALE,
                   segmented=SEGMENTED, labels=LABELS,
                   contrast_ptiles=CONTRAST_PTILES,
                   threshold_method=THRESHOLD_METHOD,
                   sholl_step_size=SHOLL_STEP_SIZE,
                   polynomial_degree=POLYNOMIAL_DEGREE,
                   save_results=True, show_logs=False)

groups.plot_avg_sholl_plot(False)
winsound.MessageBeep(0)

In [ ]:
groups.group_counts

In [ ]:
import matplotlib.pyplot as plt
plt.pie(groups.group_counts, labels=LABELS, autopct='%1.1f%%');plt.show()

In [ ]:
# All 23 Morphological features' names which will be extracted from the images
sm.ALL_FEATURE_NAMES

In [ ]:
groups.plot_feature_histograms()

  Select out of all Morphological features on which you want to perform Principal Component Analysis
- Set list of names of `pruned_features` through corresponding checkboxes of each morphological feature

In [ ]:
# @title Prune the Morphological features, if needed.
pruned_features = list(sm.ALL_FEATURE_NAMES)

def prune_features(**args):
  global pruned_features
  pruned_features = [feat for feat, val in args.items() if val]

options = dict(zip(list(sm.ALL_FEATURE_NAMES), [True] * len(sm.ALL_FEATURE_NAMES)))
_ = widgets.interact(prune_features, **options)

In [ ]:
groups.plot_feature_bar_swarm(pruned_features)

In [ ]:
groups.plot_feature_scatter_matrix(pruned_features)

In [ ]:
feature_significance, covar_matix, var_PCs = groups.pca(n_PC=6, save_results=True,
                                                        on_features=pruned_features)

In [ ]:
groups.plot_feature_significance_heatmap()

In [ ]:
groups.plot_feature_significance_vectors()

In [ ]:
%matplotlib inline
cluster_centers, clustered_data, dist = groups.get_clusters(k=3, use_features=True,
                                                            n_PC=None, plot='parallel')
print('Distribution in clusters (rows represent clusters):')
dist

In [ ]:
import numpy as np
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(aspect="equal"))
size = 0.3
vals = [v for k in dist.keys() for c, v in dist[k].iteritems()]
ilabels = [f'{k}-Cluster_{c}' for k in dist.keys() for c, v in dist[k].iteritems()]
vals = dist.to_numpy().ravel('A')
ax.pie(groups.group_counts, radius=1-size, labeldistance=1-size*2,
       wedgeprops=dict(width=size, edgecolor='w', alpha=.9), labels=LABELS)
wedges, texts, _ = ax.pie(vals, radius=1, autopct='%1.1f%%', pctdistance=1-size/2,
                       wedgeprops=dict(width=size, edgecolor='w'))
bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(ilabels[i], xy=(x, y), xytext=(1.1*np.sign(x), 1.1*y),
                horizontalalignment=horizontalalignment, **kw)

ax.set_title("Cluster distribution")

In [ ]:
feature_significance, cov_mat, var_ratios = groups.lda(n_components=3,
                                                       cluster_labels=clustered_data['cluster_label'],
                                                       on_features=pruned_features)

---